In [0]:
%tensorflow_version 2.x

In [0]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import *

%matplotlib inline

In [17]:
from google.colab import drive
drive.mount('/content/drive')
data = pd.read_csv('/content/drive/My Drive/cs230 project/collab/fer2013/fer2013.csv')

#print('Number of samples in the dataset: ', data.shape[0])
# Transform images from strings to lists of integers. TODO: use an array cast
data['pixels'] = data['pixels'].apply(lambda x: [int(pixel) for pixel in x.split()])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
emotion_cat = {0:'Angry', 1:'Disgust', 2:'Fear', 3:'Happy', 4:'Sad', 5:'Surprise', 6:'Neutral'}

# See the target distribution (check for imbalance)
#target_counts = data['emotion'].value_counts().reset_index(drop=False)
#target_counts.columns = ['emotion', 'number_samples']
#target_counts['emotion'] = target_counts['emotion'].map(emotion_cat)
#target_counts

# Select randomly 10 images
#random_seed = 1
#data_sample = data.sample(10, random_state=random_seed)
#f, axarr = plt.subplots(2, 5, figsize=(20, 10))

#i, j = 0, 0
#for idx, row in data_sample.iterrows():
#    img = np.array(row['pixels']).reshape(48,48)
#    axarr[i,j].imshow(img, cmap='gray')
#    axarr[i,j].set_title(emotion_cat[row['emotion']])
#    if j==4:
#        i += 1
#        j = 0
#    else:
#        j += 1

In [0]:
data_train = data[data['Usage']=='Training']
#print('Number samples in the training dataset: ', data_train.shape[0])

data_dev = data[data['Usage']=='PublicTest']
#print('Number samples in the development dataset: ', data_dev.shape[0])

# Retrieve train input and target
X_train, y_train = data_train['pixels'].tolist(), data_train['emotion'].values
# Reshape images to 4D (num_samples, width, height, num_channels)
X_train = np.array(X_train, dtype='float32').reshape(-1,48,48,1)
# Normalize images with max (the maximum pixel intensity is 255)
X_train = X_train/255.0

# Retrieve dev input and target
X_dev, y_dev = data_dev['pixels'].tolist(), data_dev['emotion'].values
X_dev = np.array(X_dev, dtype='float32').reshape(-1,48,48,1)
X_dev = X_dev/255.0

In [0]:
#CPNCPNCPCFF 6 layers, 21.3m params, 75% state of art acc https://arxiv.org/pdf/1612.02903.pdf
model = tf.keras.models.Sequential(   
    [
      InputLayer(input_shape=(48,48,1),name="input"),
      Conv2D(filters=64,kernel_size=5,activation='relu',padding='same',name="conv1"),
      MaxPool2D(pool_size=(2,2),name="maxpool1"),
      BatchNormalization(),
      Conv2D(filters=96,kernel_size=5,activation='relu',padding='same',name="conv2"),
      MaxPool2D(pool_size=(2,2),name="maxpool2"),
      BatchNormalization(),
      Conv2D(filters=256,kernel_size=5,activation='relu',padding='same',name="conv3"),
      MaxPool2D(pool_size=(2,2),name="maxpool3"),
      BatchNormalization(),
      Conv2D(filters=256,kernel_size=5,activation='relu',padding='same',name="conv4"),
      BatchNormalization(),
      Flatten(),
      Dense(2048,input_shape=(6*6*256,1),activation='relu',name='fc1'),
      BatchNormalization(),
      Dense(7,input_shape=(2048,1),activation='softmax',name='fc-softmax')
    ]
)

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [25]:
model.fit(X_train,y_train,batch_size=32,epochs=20,validation_data=(X_dev, y_dev))

Train on 28709 samples, validate on 3589 samples
Epoch 1/20
28709/28709 [==============================] - 16s 563us/sample - loss: 1.5161 - accuracy: 0.4478 - val_loss: 1.7416 - val_accuracy: 0.4040
Epoch 2/20
28709/28709 [==============================] - 16s 558us/sample - loss: 1.3910 - accuracy: 0.5088 - val_loss: 1.3521 - val_accuracy: 0.4739
Epoch 3/20
28709/28709 [==============================] - 16s 565us/sample - loss: 1.2641 - accuracy: 0.5579 - val_loss: 1.4997 - val_accuracy: 0.5026
Epoch 4/20
28709/28709 [==============================] - 16s 564us/sample - loss: 1.1149 - accuracy: 0.6191 - val_loss: 1.4092 - val_accuracy: 0.5155
Epoch 5/20
28709/28709 [==============================] - 16s 568us/sample - loss: 0.9588 - accuracy: 0.6819 - val_loss: 1.2840 - val_accuracy: 0.5313
Epoch 6/20
28709/28709 [==============================] - 16s 558us/sample - loss: 0.6443 - accuracy: 0.7709 - val_loss: 1.9568 - val_accuracy: 0.5587
Epoch 7/20
28709/28709 [=====================